# Notebook to analyse sale history from DVF

### Objectif 

- Pour chaque bien, avoir une visibilité des opérations réalisées autour. Récupérer le maxiumu d'informations sur cette vente.
- Identifier des opérations d'achat-revente.

In [2]:
import pandas as pd
import folium
import numpy as np
from folium.plugins import HeatMap
from tqdm import tqdm

### Chargement des données

In [3]:
df = pd.read_csv("full.csv")

/var/folders/pz/pwb4fxrn0z73q85rm8lhqkh00000gn/T/ipykernel_87899/2471715599.py:1: DtypeWarning: Columns (8,10,12,14,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv")


In [4]:
df.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [5]:
to_value_counts = ['nature_mutation', 'nombre_lots', 'code_type_local', 'type_local',  'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain']
if False : 
    for r in to_value_counts :
        print("===")
        print(df[r].value_counts())
        print()

In [6]:
df['surface_reelle_bati'].isna().mean()

np.float64(0.6918047059859841)

In [7]:
df['surface_terrain'].isna().mean()

np.float64(0.3007787989988785)

### Filtering

In [28]:
filtered_df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 'longitude', 'latitude', 'code_postal'])
filtered_df = filtered_df[filtered_df.nature_mutation == "Vente"]
# Maison ou appartement ? 
# filtered_df = filtered_df[filtered_df["type_local"].isin(["Maison", "Appartement"])] 

filtered_df = filtered_df[filtered_df['type_local'].isin(["Maison", "Appartement"])]
# localisation
# filtered_df = filtered_df[filtered_df['code_postal'].apply(lambda x: str(x).startswith('77'))]
filtered_df = filtered_df[filtered_df['code_postal'] == 77000]

In [29]:
filtered_df = filtered_df.groupby('id_mutation').agg({
    'surface_reelle_bati': 'sum',
    'nombre_pieces_principales': 'sum',
    **{col: 'first' for col in df.columns if col not in ['id_mutation', 'surface_reelle_bati', 'nombre_pieces_principales']}
}).reset_index()

In [30]:
filtered_df.shape

(259, 40)

In [31]:
filtered_df["type_local"].value_counts()

type_local
Appartement    186
Maison          73
Name: count, dtype: int64

### Affichage des données de ventes

In [45]:
# ---------------------------
# ADAPTER LE CODE AUX DONNÉES EXISTANTES
# ---------------------------

filtered_df = filtered_df[filtered_df['surface_reelle_bati'] > 0]
filtered_df['prix_m2'] = filtered_df['valeur_fonciere'] / filtered_df['surface_reelle_bati']

# 3. Coordonnées du centre de la carte (ex: Gex - 46.332212, 6.058695)
latitude_centre = filtered_df['latitude'].mean()
longitude_centre = filtered_df['longitude'].mean()

# ---------------------------
# CRÉATION DE LA CARTE
# ---------------------------

m = folium.Map(
    location=[latitude_centre, longitude_centre],
    zoom_start=14,
    tiles="cartodbpositron"
)

# Heatmap des prix/m²
if False : 
    heat_data = np.array([[row['latitude'], row['longitude'], row['prix_m2']] for _, row in filtered_df.iterrows()])
    HeatMap(
        heat_data,
        name='Prix au m²',
        #gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'},  # Personnaliser les couleurs
        min_opacity=0.1,
        radius=30
    ).add_to(m)

type_local_2_icon = {"Maison": "home",
                     "Appartement": "building"
                    }
# Marqueurs interactifs
for _, row in tqdm(filtered_df.iterrows()):
    #print(row['surface_terrain'])
           
    
    popup_html = f"""
    <b>Adresse:</b> {row['adresse_numero']} {row['adresse_nom_voie']}<br>
    <b>Date_mutation:</b>  {row['date_mutation']} <br>
    <b>Prix:</b> {row['valeur_fonciere']:,.0f} €<br>
    <b>Surface reelle bati:</b> {row['surface_reelle_bati']} <br>
    <b>Prix/m²:</b> {row['prix_m2']:,.1f} €<br>
    <a href="https://www.google.com/maps/place/{row['latitude']},{row['longitude']}/@{row['latitude']},{row['longitude']},18z" target="_blank">Vue satellite</a>
    """

    if not np.isnan(row['surface_terrain']):
        surface_terrain_html = f"<b>Surface_terrain:</b> {row['surface_terrain']:,.1f} m²<br>"
        popup_html = popup_html + surface_terrain_html



        
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_html,
        icon=folium.Icon(color = "gray" if row["type_local"] == "Appartement" else "green",  icon=type_local_2_icon[row["type_local"]], prefix='fa'),
        tooltip=f"{row['prix_m2']:,.0f} €/m²"
    ).add_to(m)

# Contrôle des calques + Affichage
print("done")
display(m)

259it [00:00, 4234.38it/s]

done


In [46]:
m.save("map.html")